In [1]:
import sys
sys.path.insert(0, '../')
from variant.varianthdf5 import VariantHDF5

%load_ext autoreload
%autoreload 2

SyntaxError: invalid syntax (varianthdf5.py, line 132)

In [3]:
VARIANT_FILE_PATH = '/home/k/Jumis/guardiome/genome-data/customers/738/variant/738.concat_snp_indel.extract_chr.recode.ann_snpeff.ann_clinvar.rename_chr.vcf.gz'
variant_hdf5 = VariantHDF5(VARIANT_FILE_PATH, reset=True)

Resetting ...
Getting data start position ...
Counting variants in chromosomes ...
	@ 1 ...
	@ 10 ...
	@ 11 ...
	@ 12 ...
	@ 13 ...
	@ 14 ...
	@ 15 ...
	@ 16 ...
	@ 17 ...
	@ 18 ...
	@ 19 ...
	@ 2 ...
	@ 20 ...
	@ 21 ...
	@ 22 ...
	@ 3 ...
	@ 4 ...
	@ 5 ...
	@ 6 ...
	@ 7 ...
	@ 8 ...
	@ 9 ...
	@ MT ...
	@ X ...
	@ Y ...
defaultdict(<function VariantHDF5._make_variant_hdf5.<locals>.<lambda> at 0x7f0b828311e0>,
            {'1': 1,
             '10': 1,
             '11': 1,
             '12': 1,
             '13': 1,
             '14': 1,
             '15': 1,
             '16': 1,
             '17': 1,
             '18': 1,
             '19': 1,
             '2': 1,
             '20': 1,
             '21': 1,
             '22': 1,
             '3': 1,
             '4': 1,
             '5': 1,
             '6': 1,
             '7': 1,
             '8': 1,
             '9': 1,
             'MT': 1,
             'X': 1,
             'Y': 1})
Making variant HDF5 ...
	@ 1 ...
		Making chrom

In [12]:
variant_hdf5.get_variant_by_id('rs662799')

[(b'A', b'11', 116792991, b'', b'rs662799', 1181.77001953125, b'G', 116792991)]